In [1]:
import sys
import os
import pandas as pd

os.chdir('/users/sbuongiorno/democracy-lab/util/')
from pyfunctions.interval_subsetter import interval_subset
from pyfunctions.parallelize_operation import parallelize_operation
from pyfunctions.str_functions import lemmatize_df_text, str_split_df_sentences
from pyfunctions.gensim_w2v_functions import export_gensim_w2v_models, w2v_embeddings, w2v_visualize_scatter_plot

os.chdir('/users/sbuongiorno/democracy-lab/word-embeddings/')
pwd = os.getcwd()

In [ ]:
hansard = pd.read_csv('~/hansard_justnine_w_year.csv')

hansard = hansard[['text','year']].copy()

hansard = hansard.rename(columns = {'text': 'speech'})

In [6]:
interval_subset(hansard, 'year', 1800, 1910, 5, fname='hansard')

In [ ]:
# run the preprocessor first 

In [ ]:
%%time
export_gensim_w2v_models('hansard_subsets/', n_cores=24)

In [93]:
#with open('propertywords_cleaned_for_word2vec.csv', newline='') as f:
#    reader = csv.reader(f)
#    keywords_list = list(reader)[1:] # skip the header

In [94]:
# keyword_list = ['evict', 'tenant', 'landlord']

In [91]:
keyword = 'tenant'

In [92]:
keyword_context = w2v_embeddings.keyword_context(pwd, keyword)

In [120]:
period_names = w2v_visualize_scatter_plot.define_periods(1800, 1915, 5)
period_words = w2v_visualize_scatter_plot.collect_text_values(keyword_context, period_names)
flat_list = w2v_visualize_scatter_plot.make_1D_list(period_words)

In [124]:
from matplotlib import cm
from numpy import linspace

colors = [ cm.gnuplot(x) for x in linspace(0, 1, len(flat_list)) ]

from adjustText import adjust_text
from matplotlib import pyplot as plt
%matplotlib inline
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap

plt.figure(figsize=(30,30), dpi = 400)

texts = []

# plt.annotate only plots one label per iteration, so we have to use a for loop 
for i in range(0,len(period_names)):    # cycle through the period names                     
    for j in range(10):     # cycle through the first ten words (you can change this variable)
        if keyword_context[i]:
            xx = period_names[i]        # on the x axis, plot the period name
            yy = [item[1] for item in keyword_context[i]][j]         # on the y axis, plot the distance -- how closely the word is related to the keyword
            txt = [item[0] for item in keyword_context[i]][j]        # grab the name of each collocated word
            colorindex = flat_list.index(txt)                     # this command keeps all dots for the same word the same color
        
        
            plt.scatter(                                           # plot dots
                xx, #x axis
                yy, # y axis
                linewidth=1, 
                color = colors[colorindex],
                s = 300, # dot size
                alpha=0.5)  # dot transparency

            # make a label for each word
            texts.append(plt.text(xx, yy, txt))

# Code to help with overlapping labels -- may take a minute to run
adjust_text(texts, force_points=0.0001, force_text=0.0035, 
                    expand_points=(1, 1), expand_text=(1, 1),
                    arrowprops=dict(arrowstyle="-", color='black', lw=0.5))


plt.xticks(rotation=90)

# Add titles
plt.title("What words were most associated with ''" + keyword + "' in the Hansard debates?", fontsize=20, fontweight=0, color='Red')
plt.xlabel("period")
plt.ylabel("similarity to " + keyword)

plt.savefig(keyword + '_' + period_names[1] + '_' + period_names[-1] +'.pdf')

plt.show()

#plt.savefig(keyword + '-over-time-' + str(startdate) + '-' + str(enddate) + '.pdf')